# Paper Simulation result analysis 

- All replicate results are saved separately, check if they share some commonality


## Section I, trajectory analysis

In [2]:
from Config import Config
from usr_func.checkfolder import checkfolder
import numpy as np
import matplotlib.pyplot as plt
import os 
from tqdm import tqdm
from matplotlib.gridspec import GridSpec
from joblib import Parallel, delayed

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 20

filepath = "./npy/temporal/"
print(os.listdir(filepath))

figpath = os.getcwd() + "/../../../../OneDrive - NTNU/MASCOT_PhD/Projects/GOOGLE/Docs/fig/Sim_2DNidelva/Simulator/AnalyticalEIBV/"
print(os.listdir(figpath))

c = Config()
plg_b = c.get_polygon_border()
plg_o = c.get_polygon_obstacle()
num_steps = c.get_num_steps()
num_replicates = c.get_num_replicates()

os.system("ls -lh " + filepath)

['R_000', 'R_001', 'R_002']
['.DS_Store', 'sigma_01', 'GRFrange700', 'up_startloc', 'GRFRange200', 'R30test']
total 0
drwxr-xr-x  3 yaolin  staff    96B Sep  5 11:41 R_000
drwxr-xr-x  3 yaolin  staff    96B Sep  5 11:41 R_001
drwxr-xr-x  3 yaolin  staff    96B Sep  5 11:41 R_002


0

In [10]:
Simulators = ["SimulatorRRTStar", "SimulatorMyopic2D"]
sigmas_nuggets = np.array([[1., .25]])
replicates = os.listdir(filepath)
Nrep = 0
for rep in replicates: 
    if rep.startswith("R_"): 
        Nrep += 1
print("Number of replicates: ", Nrep)

sigma = 1. 
nugget = .25
simulator = Simulators[0]

for i in range(Nrep):
    rep = "R_{:03d}".format(i)
    string = "/sigma_{:02d}/".format(int(10*sigma)) + "nugget_{:03d}/".format(int(100*nugget)) + simulator + "/"
    datapath = filepath + rep + string



    traj = np.empty([0, 3, num_steps, 2])
    ibv = np.empty([0, 3, num_steps])
    rmse = np.empty([0, 3, num_steps])
    vr = np.empty([0, 3, num_steps])


        r_traj = np.load(datapath + "traj.npy").reshape(1, 3, num_steps, 2)
        r_ibv = np.load(datapath + "ibv.npy").reshape(1, 3, num_steps)
        r_vr = np.load(datapath + "vr.npy").reshape(1, 3, num_steps)
        r_rmse = np.load(datapath + "rmse.npy").reshape(1, 3, num_steps)

        traj = np.append(traj, r_traj, axis=0)
        ibv = np.append(ibv, r_ibv, axis=0)
        vr = np.append(vr, r_vr, axis=0)
        rmse = np.append(rmse, r_rmse, axis=0)

    savefig = figpath[:-1] + string
    checkfolder(savefig)

    for i in tqdm(range(traj.shape[2])):
        plotf(i, "sigma: {:.1f}, nugget: {:.2f}, simulator: {:s}".format(sigma, nugget, simulator), filename=savefig + "P_{:03d}.png".format(i))


Number of replicates:  3


In [3]:
Simulators = ["SimulatorRRTStar", "SimulatorMyopic2D"]
# sigmas = [1.5, 1., .5, .1]
# nuggets = [.4, .25, .1, .01]

# sigmas = [1.]
# nuggets = [.4]

sigmas = [1.]
nuggets = [.25]

replicates = os.listdir(filepath)
Nrep = 0
for rep in replicates: 
    if rep.startswith("R_"):
        Nrep += 1
print("Number of replicates: ", Nrep)


def make_plots_total(sigma, nugget, simulator):
    
    def plotf(i, title, filename):
        fig = plt.figure(figsize=(40, 24))
        gs = GridSpec(nrows=2, ncols=3)

        def plot_subplot(data, title):
            for j in range(data.shape[0]):
                plt.plot(data[j, :i, 1], data[j, :i, 0], 'k.-', alpha=.4)
                ax.plot(plg_b[:, 1], plg_b[:, 0], 'r-.')
                ax.plot(plg_o[:, 1], plg_o[:, 0], 'r-.')
                ax.set_xlabel("East")
                ax.set_ylabel("North")
                ax.set_title(title)
                ax.set_aspect("equal")

        def plotf_sim(data, steps, ylabel="IBV"):
            N = data.shape[0]

            hx = np.arange(steps)
            ax.errorbar(hx, y=np.mean(data[:, 0, :steps], axis=0), yerr=np.std(data[:, 0, :steps], axis=0)/np.sqrt(N) * 1.645, fmt="-o", capsize=5,
                                 label="EIBV dominant")
            ax.errorbar(hx, y=np.mean(data[:, 1, :steps], axis=0), yerr=np.std(data[:, 1, :steps], axis=0)/np.sqrt(N) * 1.645, fmt="-o", capsize=5,
                                 label="IVR dominant")
            ax.errorbar(hx, y=np.mean(data[:, 2, :steps], axis=0), yerr=np.std(data[:, 2, :steps], axis=0)/np.sqrt(N) * 1.645, fmt="-o", capsize=5,
                                 label="Equal weights")
            plt.legend(loc="lower left")
            plt.xlabel('Time steps')
            plt.ylabel(ylabel)

        ax = fig.add_subplot(gs[0])
        plot_subplot(traj[:, 0, :, :], "EIBV dominant")

        ax = fig.add_subplot(gs[1])
        plot_subplot(traj[:, 1, :, :], "IVR dominant")

        ax = fig.add_subplot(gs[2])
        plot_subplot(traj[:, 2, :, :], "Equal weights")

        ax = fig.add_subplot(gs[3])
        plotf_sim(ibv, i, "IBV")

        ax = fig.add_subplot(gs[4])
        plotf_sim(vr, i, "VR")

        ax = fig.add_subplot(gs[5])
        plotf_sim(rmse, i, "RMSE")

        plt.suptitle(title)

        plt.savefig(filename)
        plt.close("all")

    string = "/sigma_{:02d}/".format(int(10*sigma)) + "nugget_{:03d}/".format(int(100*nugget)) + simulator + "/"
    traj = np.empty([0, 3, num_steps, 2])
    ibv = np.empty([0, 3, num_steps])
    rmse = np.empty([0, 3, num_steps])
    vr = np.empty([0, 3, num_steps])

    for i in range(Nrep):
        rep = "R_{:03d}".format(i)
        datapath = filepath + rep + string
        r_traj = np.load(datapath + "traj.npy").reshape(1, 3, num_steps, 2)
        r_ibv = np.load(datapath + "ibv.npy").reshape(1, 3, num_steps)
        r_vr = np.load(datapath + "vr.npy").reshape(1, 3, num_steps)
        r_rmse = np.load(datapath + "rmse.npy").reshape(1, 3, num_steps)

        traj = np.append(traj, r_traj, axis=0)
        ibv = np.append(ibv, r_ibv, axis=0)
        vr = np.append(vr, r_vr, axis=0)
        rmse = np.append(rmse, r_rmse, axis=0)

    savefig = figpath[:-1] + string
    checkfolder(savefig)

    for i in tqdm(range(traj.shape[2])):
        plotf(i, "sigma: {:.1f}, nugget: {:.2f}, simulator: {:s}".format(sigma, nugget, simulator), filename=savefig + "P_{:03d}.png".format(i))

Parallel(n_jobs=5)(delayed(make_plots_total)(sigma=sigma, nugget=nugget, simulator=simulator) for sigma in sigmas for nugget in nuggets for simulator in Simulators)


Number of replicates:  80


100%|██████████| 100/100 [02:39<00:00,  1.59s/it]


[None, None]

In [1]:
import os
import numpy as np
figpath = os.getcwd() + "/../../../../OneDrive - NTNU/MASCOT_PhD/Projects/GOOGLE/Docs/fig/Sim_2DNidelva/Simulator/"
# sigmas = [1., .1]
# nuggets = [.4, .01]


# sigmas = [1.5, 1., .5, .1]
# nuggets = [.4, .25, .1, .01]

sigmas_nuggets = np.array([[1., .25],
                           [1., .1],
                           [1.5, .25],
                           [1.5, .1],
                           [.5, .25],
                           [.5, .1],
                           [.1, .1],
                           [.1, .01]])

Simulators = ["SimulatorRRTStar", "SimulatorMyopic2D"]

# for sigma in sigmas: 
#     for nugget in nuggets: 
for sigma, nugget in sigmas_nuggets:
#     for simulator in Simulators:  
        string = "sigma_{:02d}/".format(int(10*sigma)) + "nugget_{:03d}/".format(int(100*nugget)) + "/"
        mppath = figpath + string
        os.chdir(mppath)
#             print(os.getcwd())
        os.system("ffmpeg -y -r 10 -i P_%03d.png -vcodec libx264 -crf 20 -pix_fmt yuv420p sim.mp4")
#             print(os.listdir(mppath))
#             break
#         break    
#     break

            

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.1.2_5 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack

In [4]:
mppath

'/Users/yaolin/CodeSpace/GOOGLE/Publication/src/../../../../OneDrive - NTNU/MASCOT_PhD/Projects/GOOGLE/Docs/fig/Sim_2DNidelva/Simulator/sigma_10/nugget_025/SimulatorRRTStar/'

In [6]:
os.listdir(figpath)

['.DS_Store',
 'sigma_15',
 'sigma_01',
 'ApproximateEIBV',
 'Myopic',
 'AnalyticalEIBV',
 'Pback',
 'sigma_10',
 'RRT',
 'sigma_05']